In [1]:
#%%
#IMPORTING THE DATASET
import pandas as pd
from firebase import firebase
from datetime import datetime
from pyfcm import FCMNotification
firebase_instance = firebase.FirebaseApplication('https://free-app-d2064.firebaseio.com')
data = firebase_instance.get('/', None)

In [2]:
column_list = ['ads_viewed', 
               'age_group', 
               'balance', 
               'email', 
               'gender', 
               #'interests', 
               #'location', 
               'name', 
               'password', 
               'phone', 
               'staff', 
               'token', 
               'version', 
               'video_ads_viewed']

users_token = {};faulty_users = []
# Getting the users_toekn from the users that have a token and
# returning a list of faulty_users which represent users that
# do not have a login token yet
for i in data['users'].keys():
    try:    
        data['users'][i]['token']
    
    except KeyError:
        faulty_users.append(i) 
    else:
        users_token[data['users'][i]['token']] = i

# Creating the DataFrame

App_DataFrame =  pd.DataFrame(index = users_token , columns = column_list)
App_DataFrame.fillna(0, inplace = True)


#function to fill the dataframe based on user values
def filling_dataframe(data,df,column_list):
    for i in column_list:
        if i != 'age_group':
            for j in range(len(df.index)):
                try:
                    data['users'][users_token[df.index[j]]][i]
                except KeyError:
                    continue
                else:
                    list = data['users'][users_token[df.index[j]]][i]
                    for k in df.columns:
                        if k == i:
                            df.loc[df.index[j],k] = data['users'][users_token[df.index[j]]][i]
        else:
            for l in range(len(df.index)):
                try:
                    data['users'][users_token[df.index[l]]]["age_group"]
                except KeyError:
                    continue
                else:
                    string = data['users'][users_token[df.index[l]]]["age_group"]
                    integer = int(string[6:])
                    currentYear = int(datetime.now().year)
                    currentAge = currentYear - integer
                    df.loc[df.index[l],i] = currentAge
    
    #replacing the token column with the app_id 
    df['token'].apply(lambda x : users_token[x])
    df.token.name = 'App_id'
    
    return df


App_DataFrame = filling_dataframe(data,App_DataFrame,column_list)

In [3]:
version_1_users = App_DataFrame[App_DataFrame['version'] == 'version 1.0.2']
version_2_users = App_DataFrame[App_DataFrame['version'] == 'version 2']

In [4]:
def pushing_notification_user_version1(df): 
    for i in df.index:
        push_service = FCMNotification(api_key="AAAAhCYz34A:APA91bG9X7-rVAGqt3IQhZZ_gV7vJF0GLQ7KonHbB1guslcmUyx9Xa_UltDye-LqnwRryB9zJd1bBlDKPQTv5KZRuW1iT__VgQ1N3qpiJdMqwtGkd9F9zTLMhAyf-ect8B0Dw5nF8UrwJT7778YU7_ZnxidtVZpg2Q")
        registration_id = str(df.loc[i,'token'])
        message_title = "Ads available"
        message_body = "Hi {}, we just sent you and ad, but you need to download the updated version of this app to view it. Adverts are better experienced on the latest version of our app. Download the update at www.freetek.com.ng. No dull yourself o!".format(df.loc[i,'name'])
        result = push_service.notify_single_device(registration_id=registration_id, message_title=message_title, message_body=message_body)
   
    return result


def pushing_notification_user_version2(df): 
    for i in df.index:
        push_service = FCMNotification(api_key="AAAAhCYz34A:APA91bG9X7-rVAGqt3IQhZZ_gV7vJF0GLQ7KonHbB1guslcmUyx9Xa_UltDye-LqnwRryB9zJd1bBlDKPQTv5KZRuW1iT__VgQ1N3qpiJdMqwtGkd9F9zTLMhAyf-ect8B0Dw5nF8UrwJT7778YU7_ZnxidtVZpg2Q")
        registration_id = str(df.loc[i,'token'])
        message_title = "Ads available"
        message_body = "Hi {}, we just sent you and ad. It's time to earn some freecoin!".format(df.loc[i,'name'])
        result = push_service.notify_single_device(registration_id=registration_id, message_title=message_title, message_body=message_body)
    
    return result


pushing_notification_user_version1(version_1_users)
pushing_notification_user_version2(version_2_users)

{'multicast_ids': [4806339500701520050],
 'success': 1,
 'failure': 0,
 'canonical_ids': 0,
 'results': [{'message_id': '0:1569628717722658%e00a9d74e00a9d74'}],
 'topic_message_id': None}

# MISCELLANEOUS

In [5]:
#returns the leaderboard for the freecoin challenge
App_DataFrame['balance'] = App_DataFrame['balance'].apply(lambda x: int(x))
App_DataFrame.sort_values(by='balance', ascending=False)[:6]

,ads_viewed,age_group,balance,email,gender,name,password,phone,staff,token,version,video_ads_viewed
ea5h3J4ov9I:APA91bFZ5OtvDxbc8-wZ8O7IVEhbCFJW0j2FpN-pzJbgBUtRkLAHswCxWBl-0-FDrIiIND7tbLHmxTWV8DRciWoNo3TGDAezivSF5VzSIXaU2BItl7gRWiL_57gWn7B_yaBMCOIp2ADw,22,0,91,freetek.app@gmail.com,male,Adewole test,1996july,08129151589,false,ea5h3J4ov9I:APA91bFZ5OtvDxbc8-wZ8O7IVEhbCFJW0j...,version 2,0
dHTWRfTETwU:APA91bEjceQfxD_a5SGUUMJPEB0n91fw-Aw_oeJcUWzr0LnQaKdkQW_zop67Vhve8r_TUaQIuL1YpWKW13I7VyMhqhmjDVwJfQ-4PC9fzCLzaYxW7ZehbySGyJawxdkmZLntClNzvnlH,5,27,68,idinellalek@gmail.com,female,Lekwa Chidinma,aliji2000,08168276979,false,dHTWRfTETwU:APA91bEjceQfxD_a5SGUUMJPEB0n91fw-A...,version 2,0
fa2Cs5qCYJo:APA91bGA1M9Ad5jVLP1bCcPEk6eZzI5R7URoHujZlhs0qfjL3foBSSqeDWVKYM_1YLhJuGU0Yv1oW8aZzIr7ImKrE_aoW1-_otb1sjeEqrlA9QDMX_QWxvJjAftAGA9CCStMo8ZBA-Pj,9,24,44,Yettybabe19952@gmail.com,female,Oyelami Yetunde Oyelami,09038323476,09038323476,false,fa2Cs5qCYJo:APA91bGA1M9Ad5jVLP1bCcPEk6eZzI5R7U...,version 2,0
feQKdrToXQA:APA91bFT0slhbs0D-gweGi-BWp4M_QGWM3na0EjbU9HpHCLp64eFOhJia5H5ZwMWtWGXbBfWPqo80e7T7byQltwt4tX_P4Ho5PdB61AORsUBAe7UWTEM7StEy9CwTw-uQ6HIE3HRfAwu,8,25,41,krisjuco55@gmail.com,male,Udemba Chinonso,Ubiquitous,08029996193,false,feQKdrToXQA:APA91bFT0slhbs0D-gweGi-BWp4M_QGWM3...,version 2,0
e-Zxcoz_-SY:APA91bHUv7GWbSC49smrm9F6X8hd9FiZNLygS-RlgVXXBWWLnigqmi5iW_yzuDXGdK8TYR2ACTKederQ2B7oANng8eHNfMQC5qugtc2hb0RRbJDwdZjj7Mc73Z_nup3d4S1zSFCRa1dw,7,20,39,orlamethey1@gmail.com,male,Ifederu Olamide,Joshuadmighty2029,08180197495,false,e-Zxcoz_-SY:APA91bHUv7GWbSC49smrm9F6X8hd9FiZNL...,version 2,0
cb-FRq5KyKM:APA91bGT_xsSGgol5ZWi7RBhJuoxEAXDPzM6t5mepHqkFkkfh7NOjFh0v3ZbiJfW90yzXVvStVIAfth4V2dBrqLsyu8lZpqoZUxgR5vzbEch0AXpsA-phY71KY9LT2PDS5N_FYY-pH8m,6,25,38,solufemi272@gmail.com,male,Isaac Olufemi,femot94,09093635695,false,cb-FRq5KyKM:APA91bGT_xsSGgol5ZWi7RBhJuoxEAXDPz...,version 2,0
